In [ ]:
#The following code runs veryyyyy slow. There is certainly room for improvement in both its efficiency and the detection method used.
#The pothole detection is done using basic image processing and the underlying idea was one that struck my mind right after reading the question and so, it may be quite unorthodox/weird.

import cv2
import numpy as np

def detectt(image): #pothole detecting function 
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret, th1 = cv2.threshold(gray, 200, 255, cv2.THRESH_TOZERO)
    ret, th2 = cv2.threshold(th1, 120, 255, cv2.THRESH_BINARY) #thresholds to clean up the image
    edges = cv2.Canny(th2,50,150,apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100,minLineLength=5, maxLineGap=10) #lane detection 
    maxx=0;minx=1279
    maxline=[];minline=[];
    for points in lines: #loop to find the outer most (bounding) lane lines
        x1,y1,x2,y2=points[0]
        if x2>maxx:
            maxline=points[0];maxx=x2
        if x1<minx:
            minline=points[0];minx=x1
    maxline[0]=maxline[0]-40;maxline[2]=maxline[2]-40;
    minline[0]=minline[0]+40;minline[2]=minline[2]+40;
    for i in range(1279): #double loop that makes all pixels outside the lane lines equal to zero
        for j in range(593):
            if (j-maxline[1] < ((maxline[3]-maxline[1])/(maxline[2]-maxline[0]))*(i-maxline[0])) or (j-minline[1] < ((minline[3]-minline[1])/(minline[2]-minline[0]))*(i-minline[0])):
                edges[j][i]=0
            if j>800:
                edges[j][i]=0
    M = cv2.moments(edges) #finds the geometric center of all remaiing pixels (that have not been removed by the prev loop)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    cv2.rectangle(image, (cX+45, cY+45), (cX-45, cY-45), (255,0,0), 2)
    cv2.imshow("Image", image) #dyamically shows the image with a bounding box around the pothole


cap = cv2.VideoCapture('bolt_test_pothole.mp4') #passes each frame of the video to the pothole detection function
while(cap.isOpened()):
    _, frame = cap.read()
    detectt(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()